In [50]:
# Dependencies
from flask import Flask, render_template, redirect, jsonify

import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, func, inspect, or_
import pandas as pd
import geojson
import os
import json
import geopandas as gpd
from shapely.geometry import Point, Polygon
from flask_marshmallow import Marshmallow

In [51]:
app = Flask(__name__)

database_path = '../flask_app/static/data/data_all.sqlite'
engine = create_engine(f'sqlite:///{database_path}')
conn = engine.connect()

# Create our session (link) from Python to the DB
session = Session(engine)

In [52]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [53]:
# View all of the classes that automap found
Base.classes.keys()

['ppa', 'weather']

In [54]:
# Save references to each table
Parking = Base.classes.ppa
Weather = Base.classes.weather

In [55]:
inspector = inspect(engine)
columns = inspector.get_columns('ppa')
for column in columns:
    print(column["name"], column["type"])

anon_ticket_number INTEGER
issue_datetime DATETIME
state VARCHAR(20)
anon_plate_id VARCHAR(50)
location VARCHAR(250)
violation_desc VARCHAR(100)
fine INTEGER
issuing_agency VARCHAR(100)
lat FLOAT
lon FLOAT
zip_code VARCHAR(100)
month INTEGER
day INTEGER
hour INTEGER
ymdh VARCHAR(100)


In [56]:
inspector = inspect(engine)
columns = inspector.get_columns('weather')
for column in columns:
    print(column["name"], column["type"])

dt DATETIME
lat FLOAT
lon FLOAT
temp FLOAT
feels_like FLOAT
temp_min FLOAT
temp_max FLOAT
humidity INTEGER
wind_speed FLOAT
wind_deg INTEGER
rain_1h FLOAT
rain_3h FLOAT
snow_1h FLOAT
snow_3h FLOAT
clouds_all INTEGER
weather_id VARCHAR(20)
weather_main VARCHAR(30)
weather_description VARCHAR(300)
weather_icon VARCHAR(20)
month INTEGER
day INTEGER
hour INTEGER
ymdh VARCHAR(100)


### Heatmap

In [8]:
# For heatmap...
# ---------------
coordinates = session.query(Parking.lat, Parking.lon).all()
coordinates

[(40.07556801098219, -75.20566054077817),
 (39.94454265789941, -75.17269423825759),
 (39.94375773860189, -75.16637856084623),
 (39.94375773860189, -75.16637856084623),
 (39.95579417425459, -75.19423128900449),
 (40.0362783210142, -75.1752040391761),
 (39.94058336152956, -75.14963006094659),
 (39.92136, -75.16656),
 (39.940235385361994, -75.14948786474339),
 (39.97095415719344, -75.18032190494884),
 (39.953470922600815, -75.17238597738036),
 (39.95781432816961, -75.16332084492038),
 (39.96011076503564, -75.16608250437787),
 (39.96011076503564, -75.16608250437787),
 (39.95781432816961, -75.16332084492038),
 (39.95350717231464, -75.19770632424232),
 (39.954837, -75.21005799999999),
 (39.92477059124093, -75.17458307654671),
 (39.92477059124093, -75.17458307654671),
 (39.94343006094904, -75.16374355657506),
 (39.97192719737674, -75.13296424339006),
 (39.96061554140601, -75.14195634900355),
 (39.950605510548975, -75.16814185331755),
 (40.07556801098219, -75.20566054077817),
 (40.077010000392

In [14]:
coordinates_list = []

for pair in coordinates:
    coordinates_list.append(list(pair))
    
coordinates_list

[[40.07556801098219, -75.20566054077817],
 [39.94454265789941, -75.17269423825759],
 [39.94375773860189, -75.16637856084623],
 [39.94375773860189, -75.16637856084623],
 [39.95579417425459, -75.19423128900449],
 [40.0362783210142, -75.1752040391761],
 [39.94058336152956, -75.14963006094659],
 [39.92136, -75.16656],
 [39.940235385361994, -75.14948786474339],
 [39.97095415719344, -75.18032190494884],
 [39.953470922600815, -75.17238597738036],
 [39.95781432816961, -75.16332084492038],
 [39.96011076503564, -75.16608250437787],
 [39.96011076503564, -75.16608250437787],
 [39.95781432816961, -75.16332084492038],
 [39.95350717231464, -75.19770632424232],
 [39.954837, -75.21005799999999],
 [39.92477059124093, -75.17458307654671],
 [39.92477059124093, -75.17458307654671],
 [39.94343006094904, -75.16374355657506],
 [39.97192719737674, -75.13296424339006],
 [39.96061554140601, -75.14195634900355],
 [39.950605510548975, -75.16814185331755],
 [40.07556801098219, -75.20566054077817],
 [40.077010000392

In [10]:
type(coordinates[0])

sqlalchemy.util._collections.result

In [15]:
response = {}
response['heatmap_coordinates'] = coordinates_list
response

{'heatmap_coordinates': [[40.07556801098219, -75.20566054077817],
  [39.94454265789941, -75.17269423825759],
  [39.94375773860189, -75.16637856084623],
  [39.94375773860189, -75.16637856084623],
  [39.95579417425459, -75.19423128900449],
  [40.0362783210142, -75.1752040391761],
  [39.94058336152956, -75.14963006094659],
  [39.92136, -75.16656],
  [39.940235385361994, -75.14948786474339],
  [39.97095415719344, -75.18032190494884],
  [39.953470922600815, -75.17238597738036],
  [39.95781432816961, -75.16332084492038],
  [39.96011076503564, -75.16608250437787],
  [39.96011076503564, -75.16608250437787],
  [39.95781432816961, -75.16332084492038],
  [39.95350717231464, -75.19770632424232],
  [39.954837, -75.21005799999999],
  [39.92477059124093, -75.17458307654671],
  [39.92477059124093, -75.17458307654671],
  [39.94343006094904, -75.16374355657506],
  [39.97192719737674, -75.13296424339006],
  [39.96061554140601, -75.14195634900355],
  [39.950605510548975, -75.16814185331755],
  [40.0755680

In [20]:
type(response['heatmap_coordinates'][0])

list

### Scatter plot

In [57]:
# Get data for scatter plot...
# ------------------
data_per_hour = session.query(Weather.dt, Weather.feels_like, Weather.humidity,
                              Weather.rain_1h, Weather.snow_3h, Weather.weather_description,
                              func.count(Parking.anon_ticket_number), func.avg(Parking.fine)).\
                        filter(Parking.ymdh == Weather.ymdh).\
                        group_by(Parking.ymdh).\
                        order_by(Weather.dt.asc()).all()

In [58]:
data_per_hour

[(datetime.datetime(2017, 1, 1, 0, 0),
  27.28,
  42,
  0.0,
  0.0,
  'overcast clouds',
  2,
  171.0),
 (datetime.datetime(2017, 1, 1, 1, 0),
  30.25,
  46,
  0.0,
  0.0,
  'broken clouds',
  2,
  51.0),
 (datetime.datetime(2017, 1, 1, 2, 0),
  30.16,
  46,
  0.0,
  0.0,
  'overcast clouds',
  1,
  51.0),
 (datetime.datetime(2017, 1, 1, 3, 0),
  30.34,
  45,
  0.0,
  0.0,
  'overcast clouds',
  1,
  51.0),
 (datetime.datetime(2017, 1, 1, 5, 0),
  30.54,
  43,
  0.0,
  0.0,
  'overcast clouds',
  2,
  51.0),
 (datetime.datetime(2017, 1, 1, 6, 0),
  30.54,
  43,
  0.0,
  0.0,
  'overcast clouds',
  2,
  63.5),
 (datetime.datetime(2017, 1, 1, 7, 0),
  31.15,
  45,
  0.0,
  0.0,
  'overcast clouds',
  3,
  67.66666666666667),
 (datetime.datetime(2017, 1, 1, 8, 0),
  31.93,
  46,
  0.0,
  0.0,
  'scattered clouds',
  12,
  68.5),
 (datetime.datetime(2017, 1, 1, 9, 0),
  32.18,
  52,
  0.0,
  0.0,
  'broken clouds',
  8,
  73.5),
 (datetime.datetime(2017, 1, 1, 10, 0),
  31.12,
  50,
  0.0,

In [59]:
print(len(data_per_hour))
type(data_per_hour[0]) # See the returned data type

8397


sqlalchemy.util._collections.result

In [60]:
# Conver 'sqlalchemy.util._collections.result' into 'list' type
data_per_hour_list = []

for data in data_per_hour:
    data_per_hour_list.append(list(data))
    
data_per_hour_list

[[datetime.datetime(2017, 1, 1, 0, 0),
  27.28,
  42,
  0.0,
  0.0,
  'overcast clouds',
  2,
  171.0],
 [datetime.datetime(2017, 1, 1, 1, 0),
  30.25,
  46,
  0.0,
  0.0,
  'broken clouds',
  2,
  51.0],
 [datetime.datetime(2017, 1, 1, 2, 0),
  30.16,
  46,
  0.0,
  0.0,
  'overcast clouds',
  1,
  51.0],
 [datetime.datetime(2017, 1, 1, 3, 0),
  30.34,
  45,
  0.0,
  0.0,
  'overcast clouds',
  1,
  51.0],
 [datetime.datetime(2017, 1, 1, 5, 0),
  30.54,
  43,
  0.0,
  0.0,
  'overcast clouds',
  2,
  51.0],
 [datetime.datetime(2017, 1, 1, 6, 0),
  30.54,
  43,
  0.0,
  0.0,
  'overcast clouds',
  2,
  63.5],
 [datetime.datetime(2017, 1, 1, 7, 0),
  31.15,
  45,
  0.0,
  0.0,
  'overcast clouds',
  3,
  67.66666666666667],
 [datetime.datetime(2017, 1, 1, 8, 0),
  31.93,
  46,
  0.0,
  0.0,
  'scattered clouds',
  12,
  68.5],
 [datetime.datetime(2017, 1, 1, 9, 0),
  32.18,
  52,
  0.0,
  0.0,
  'broken clouds',
  8,
  73.5],
 [datetime.datetime(2017, 1, 1, 10, 0),
  31.12,
  50,
  0.0,

In [14]:
# Example syntax......

# t = [[1,2,3], [4,5,6]]
# keys = ['a', 'b', 'c']
# d = [dict(zip(keys, l)) for l in t ]
# d

In [15]:
# Transform our response data from a list of lists to a list of dictionaries
keys = ['datetime', 'temp_feels_like', 'humidity', 'rain_1h', 'snow_3h',
        'weather_description', 'total_ticket_number', 'fine']

data_per_hour_transformed = [dict(zip(keys, l)) for l in data_per_hour_list]
data_per_hour_transformed

[{'datetime': datetime.datetime(2017, 1, 1, 0, 0),
  'temp_feels_like': 27.28,
  'humidity': 42,
  'rain_1h': 0.0,
  'snow_3h': 0.0,
  'weather_description': 'overcast clouds',
  'total_ticket_number': 2,
  'fine': 171.0},
 {'datetime': datetime.datetime(2017, 1, 1, 1, 0),
  'temp_feels_like': 30.25,
  'humidity': 46,
  'rain_1h': 0.0,
  'snow_3h': 0.0,
  'weather_description': 'broken clouds',
  'total_ticket_number': 2,
  'fine': 51.0},
 {'datetime': datetime.datetime(2017, 1, 1, 2, 0),
  'temp_feels_like': 30.16,
  'humidity': 46,
  'rain_1h': 0.0,
  'snow_3h': 0.0,
  'weather_description': 'overcast clouds',
  'total_ticket_number': 1,
  'fine': 51.0},
 {'datetime': datetime.datetime(2017, 1, 1, 3, 0),
  'temp_feels_like': 30.34,
  'humidity': 45,
  'rain_1h': 0.0,
  'snow_3h': 0.0,
  'weather_description': 'overcast clouds',
  'total_ticket_number': 1,
  'fine': 51.0},
 {'datetime': datetime.datetime(2017, 1, 1, 5, 0),
  'temp_feels_like': 30.54,
  'humidity': 43,
  'rain_1h': 0.0

In [76]:
# ----------------------!!!
# Potential work around for the Heroku error "TypeError: Object of type Decimal is not JSON serializable"...

response_list = []
for data in data_per_hour_transformed:
    response_dict = {}
    response_dict['datetime'] = data['datetime']
    response_dict['temp_feels_like'] = str(data['temp_feels_like'])
    response_dict['humidity'] = str(data['humidity'])
    response_dict['rain_1h'] = str(data['rain_1h'])
    response_dict['snow_3h'] = str(data['snow_3h'])
    response_dict['weather_description'] = data['weather_description']
    response_dict['total_ticket_number'] = str(data['total_ticket_number'])
    response_dict['fine'] = str(data['fine'])
    response_list.append(response_dict)

response_list     

[{'datetime': datetime.datetime(2017, 1, 1, 0, 0),
  'temp_feels_like': '27.28',
  'humidity': '42',
  'rain_1h': '0.0',
  'snow_3h': '0.0',
  'weather_description': 'overcast clouds',
  'total_ticket_number': '2',
  'fine': '171.0'},
 {'datetime': datetime.datetime(2017, 1, 1, 1, 0),
  'temp_feels_like': '30.25',
  'humidity': '46',
  'rain_1h': '0.0',
  'snow_3h': '0.0',
  'weather_description': 'broken clouds',
  'total_ticket_number': '2',
  'fine': '51.0'},
 {'datetime': datetime.datetime(2017, 1, 1, 2, 0),
  'temp_feels_like': '30.16',
  'humidity': '46',
  'rain_1h': '0.0',
  'snow_3h': '0.0',
  'weather_description': 'overcast clouds',
  'total_ticket_number': '1',
  'fine': '51.0'},
 {'datetime': datetime.datetime(2017, 1, 1, 3, 0),
  'temp_feels_like': '30.34',
  'humidity': '45',
  'rain_1h': '0.0',
  'snow_3h': '0.0',
  'weather_description': 'overcast clouds',
  'total_ticket_number': '1',
  'fine': '51.0'},
 {'datetime': datetime.datetime(2017, 1, 1, 5, 0),
  'temp_feels_

In [79]:
# Test new object for jsonify returns string values
response_list[0]['temp_feels_like']

'27.28'

In [80]:
# Old object for jsonify should still return float values
data_per_hour_transformed[0]['temp_feels_like']

27.28

In [16]:
# Code not used....
# ----------------------

response = {}
response['datetime'] = []
response['temp_feels_like'] = []
response['humidity'] = []
response['rain_1h'] = []
response['snow_3h'] = []
response['weather_description'] = []
response['total_ticket_number'] = []
response['fine'] = []

for i in data_per_hour:
    response['datetime'].append(i[0])
    response['temp_feels_like'].append(i[1])
    response['humidity'].append(i[2])
    response['rain_1h'].append(i[3])
    response['snow_3h'].append(i[4])
    response['weather_description'].append(i[5])
    response['total_ticket_number'].append(i[6])
    response['fine'].append(i[7])

In [17]:
response

{'datetime': [datetime.datetime(2017, 1, 1, 0, 0),
  datetime.datetime(2017, 1, 1, 1, 0),
  datetime.datetime(2017, 1, 1, 2, 0),
  datetime.datetime(2017, 1, 1, 3, 0),
  datetime.datetime(2017, 1, 1, 5, 0),
  datetime.datetime(2017, 1, 1, 6, 0),
  datetime.datetime(2017, 1, 1, 7, 0),
  datetime.datetime(2017, 1, 1, 8, 0),
  datetime.datetime(2017, 1, 1, 9, 0),
  datetime.datetime(2017, 1, 1, 10, 0),
  datetime.datetime(2017, 1, 1, 11, 0),
  datetime.datetime(2017, 1, 1, 12, 0),
  datetime.datetime(2017, 1, 1, 13, 0),
  datetime.datetime(2017, 1, 1, 15, 0),
  datetime.datetime(2017, 1, 1, 16, 0),
  datetime.datetime(2017, 1, 1, 17, 0),
  datetime.datetime(2017, 1, 1, 18, 0),
  datetime.datetime(2017, 1, 1, 20, 0),
  datetime.datetime(2017, 1, 1, 23, 0),
  datetime.datetime(2017, 1, 2, 0, 0),
  datetime.datetime(2017, 1, 2, 1, 0),
  datetime.datetime(2017, 1, 2, 2, 0),
  datetime.datetime(2017, 1, 2, 3, 0),
  datetime.datetime(2017, 1, 2, 4, 0),
  datetime.datetime(2017, 1, 2, 6, 0),
  d

### Marker by violation type

In [18]:
violation_types = session.query(Parking.lat, Parking.lon, Parking.issue_datetime, Parking.violation_desc,
                              Parking.location, Parking.fine, Parking.issuing_agency).\
                              filter(or_(Parking.violation_desc == 'UNREG/ABANDONED VEH',
                                     Parking.violation_desc == 'STOP/BLOCK HIGHWAY',
                                     Parking.violation_desc == 'BLOCKNG MASS TRANSIT',
                                     Parking.violation_desc == 'PARKED ON GRASS',
                                     Parking.violation_desc == 'EXCESSIVE NOISE')).all()
violation_types

[(40.009945000798346,
  -75.17408535661869,
  datetime.datetime(2017, 2, 10, 12, 14),
  'UNREG/ABANDONED VEH',
  '2800 FOX ST',
  301,
  'POLICE'),
 (39.93812841531151,
  -75.2185607884124,
  datetime.datetime(2017, 3, 1, 16, 45),
  'UNREG/ABANDONED VEH',
  '1621 S 53RD ST',
  301,
  'POLICE'),
 (39.959517248553894,
  -75.205249443385,
  datetime.datetime(2017, 2, 26, 22, 37),
  'BLOCKNG MASS TRANSIT',
  '311 N 41ST ST',
  101,
  'SEPTA'),
 (40.01298827497849,
  -75.15717431145819,
  datetime.datetime(2017, 2, 21, 16, 15),
  'UNREG/ABANDONED VEH',
  '3856 PULASKI AVE',
  301,
  'POLICE'),
 (40.038548855854025,
  -75.05546195365335,
  datetime.datetime(2017, 2, 14, 2, 0),
  'UNREG/ABANDONED VEH',
  '2903 TYSON AVE',
  301,
  'POLICE'),
 (39.982633893674326,
  -75.12826626292231,
  datetime.datetime(2017, 1, 13, 8, 23),
  'STOP/BLOCK HIGHWAY',
  '2400 AMBER ST',
  51,
  'POLICE'),
 (40.044513162158495,
  -75.16119990203995,
  datetime.datetime(2017, 1, 11, 19, 55),
  'UNREG/ABANDONED VEH

In [19]:
# Conver 'sqlalchemy.util._collections.result' into 'list' type
violation_types_list = []

for data in violation_types:
    violation_types_list.append(list(data))
    
violation_types_list

[[40.009945000798346,
  -75.17408535661869,
  datetime.datetime(2017, 2, 10, 12, 14),
  'UNREG/ABANDONED VEH',
  '2800 FOX ST',
  301,
  'POLICE'],
 [39.93812841531151,
  -75.2185607884124,
  datetime.datetime(2017, 3, 1, 16, 45),
  'UNREG/ABANDONED VEH',
  '1621 S 53RD ST',
  301,
  'POLICE'],
 [39.959517248553894,
  -75.205249443385,
  datetime.datetime(2017, 2, 26, 22, 37),
  'BLOCKNG MASS TRANSIT',
  '311 N 41ST ST',
  101,
  'SEPTA'],
 [40.01298827497849,
  -75.15717431145819,
  datetime.datetime(2017, 2, 21, 16, 15),
  'UNREG/ABANDONED VEH',
  '3856 PULASKI AVE',
  301,
  'POLICE'],
 [40.038548855854025,
  -75.05546195365335,
  datetime.datetime(2017, 2, 14, 2, 0),
  'UNREG/ABANDONED VEH',
  '2903 TYSON AVE',
  301,
  'POLICE'],
 [39.982633893674326,
  -75.12826626292231,
  datetime.datetime(2017, 1, 13, 8, 23),
  'STOP/BLOCK HIGHWAY',
  '2400 AMBER ST',
  51,
  'POLICE'],
 [40.044513162158495,
  -75.16119990203995,
  datetime.datetime(2017, 1, 11, 19, 55),
  'UNREG/ABANDONED VEH

In [20]:
# Transform our response data from a list of lists to a list of dictionaries
keys = ['lat', 'lon', 'issue_datetime', 'violation_desc', 'location', 'fine', 'issuing_agency']

violation_types_transformed = [dict(zip(keys, l)) for l in violation_types_list]
violation_types_transformed

[{'lat': 40.009945000798346,
  'lon': -75.17408535661869,
  'issue_datetime': datetime.datetime(2017, 2, 10, 12, 14),
  'violation_desc': 'UNREG/ABANDONED VEH',
  'location': '2800 FOX ST',
  'fine': 301,
  'issuing_agency': 'POLICE'},
 {'lat': 39.93812841531151,
  'lon': -75.2185607884124,
  'issue_datetime': datetime.datetime(2017, 3, 1, 16, 45),
  'violation_desc': 'UNREG/ABANDONED VEH',
  'location': '1621 S 53RD ST',
  'fine': 301,
  'issuing_agency': 'POLICE'},
 {'lat': 39.959517248553894,
  'lon': -75.205249443385,
  'issue_datetime': datetime.datetime(2017, 2, 26, 22, 37),
  'violation_desc': 'BLOCKNG MASS TRANSIT',
  'location': '311 N 41ST ST',
  'fine': 101,
  'issuing_agency': 'SEPTA'},
 {'lat': 40.01298827497849,
  'lon': -75.15717431145819,
  'issue_datetime': datetime.datetime(2017, 2, 21, 16, 15),
  'violation_desc': 'UNREG/ABANDONED VEH',
  'location': '3856 PULASKI AVE',
  'fine': 301,
  'issuing_agency': 'POLICE'},
 {'lat': 40.038548855854025,
  'lon': -75.0554619536

In [21]:
data_per_hour = session.query(Parking.violation_desc, Weather.feels_like, Weather.humidity,
                              Weather.rain_1h, Weather.snow_3h, Weather.weather_description,
                              func.count(Parking.anon_ticket_number), func.avg(Parking.fine)).\
                        filter(Parking.ymdh == Weather.ymdh).\
                        group_by(Parking.ymdh).\
                        order_by(Weather.dt.asc()).all()
data_per_hour

[('HP RESERVED SPACE', 27.28, 42, 0.0, 0.0, 'overcast clouds', 2, 171.0),
 ('DOUBLE PARKED', 30.25, 46, 0.0, 0.0, 'broken clouds', 2, 51.0),
 ('CORNER CLEARANCE', 30.16, 46, 0.0, 0.0, 'overcast clouds', 1, 51.0),
 ('SIDEWALK', 30.34, 45, 0.0, 0.0, 'overcast clouds', 1, 51.0),
 ('SIDEWALK', 30.54, 43, 0.0, 0.0, 'overcast clouds', 2, 51.0),
 ('SIDEWALK', 30.54, 43, 0.0, 0.0, 'overcast clouds', 2, 63.5),
 ('FIRE HYDRANT',
  31.15,
  45,
  0.0,
  0.0,
  'overcast clouds',
  3,
  67.66666666666667),
 ('EXPIRED TAG', 31.93, 46, 0.0, 0.0, 'scattered clouds', 12, 68.5),
 ('PARKING PROHBITED', 32.18, 52, 0.0, 0.0, 'broken clouds', 8, 73.5),
 ('FIRE HYDRANT',
  31.12,
  50,
  0.0,
  0.0,
  'overcast clouds',
  3,
  52.666666666666664),
 ('SIDEWALK', 31.62, 50, 0.0, 0.0, 'broken clouds', 2, 46.0),
 ('BLOCKING DRIVEWAY', 27.57, 52, 0.0, 0.0, 'few clouds', 1, 51.0),
 ('LOADING ZONE', 30.56, 57, 0.0, 0.0, 'scattered clouds', 1, 31.0),
 ('DOUBLE PARKED', 37.0, 51, 0.0, 0.0, 'scattered clouds', 1, 51.

In [22]:
check_ppa_info = pd.read_sql_query('select * from ppa', con=engine)
check_ppa_info

,anon_ticket_number,issue_datetime,state,anon_plate_id,location,violation_desc,fine,issuing_agency,lat,lon,zip_code,month,day,hour,ymdh
0,7689050,2017-01-09 12:29:00,PA,1975860,8419 GERMANTOWN AVE,METER EXPIRED,26,PPA,40.075568,-75.205661,19118,1,9,12,2017-01-09 12
1,7689051,2017-01-10 14:34:00,FL,4338399,1800 SOUTH ST,METER EXPIRED CC,36,PPA,39.944543,-75.172694,19146,1,10,14,2017-01-10 14
2,7689052,2017-01-10 14:53:00,PA,4338400,1430 SOUTH ST,METER EXPIRED CC,36,PPA,39.943758,-75.166379,19146,1,10,14,2017-01-10 14
3,7689053,2017-01-10 14:55:00,PA,4338400,1430 SOUTH ST,EXPIRED INSPECTION,41,PPA,39.943758,-75.166379,19146,1,10,14,2017-01-10 14
4,7689055,2017-01-10 12:04:00,PA,4338402,1 S 36TH ST,METER EXPIRED CC,36,PPA,39.955794,-75.194231,19104,1,10,12,2017-01-10 12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430126,9415761,2017-12-20 10:39:00,PA,1295679,1000 ARCH ST,PARKING PROHBITED CC,51,PPA,39.953932,-75.157692,19107,12,20,10,2017-12-20 10
430127,9415762,2017-12-19 14:49:00,PA,1357915,100 E CHELTEN AVE,METER EXPIRED,26,PPA,40.037772,-75.172608,19144,12,19,14,2017-12-19 14
430128,9415763,2017-12-20 09:04:00,PA,503673,2900 N 5TH ST,EXPIRED INSPECTION,41,PPA,39.995958,-75.139382,19133,12,20,9,2017-12-20 09
430129,9415766,2017-12-20 09:41:00,PA,623934,800 SPRUCE ST,HP RESERVED SPACE,301,PPA,39.945860,-75.155208,19107,12,20,9,2017-12-20 09


##### Zip codes - Will not use...

In [23]:
geojson_path = '../resources/zipcodes_poly.geojson'
# geojson_path = os.path.join('..', 'resources', 'zipcodes_poly.geojson')

In [24]:
zip_df = gpd.read_file(geojson_path)
zip_df.head()

,OBJECTID,CODE,COD,Shape__Area,Shape__Length,geometry
0,1,19120,20,9.177970e+07,49921.544063,"POLYGON ((-75.11107 40.04682, -75.10943 40.045..."
1,2,19121,21,6.959879e+07,39534.887217,"POLYGON ((-75.19227 39.99463, -75.19205 39.994..."
2,3,19122,22,3.591632e+07,24124.645221,"POLYGON ((-75.15406 39.98601, -75.15328 39.985..."
3,4,19123,23,3.585175e+07,26421.728982,"POLYGON ((-75.15190 39.97056, -75.15150 39.970..."
4,5,19124,24,1.448080e+08,63658.770420,"POLYGON ((-75.09660 40.04249, -75.09281 40.039..."


In [25]:
zip_json = zip_df.to_json()
zip_json

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"COD": 20, "CODE": "19120", "OBJECTID": 1, "Shape__Area": 91779697.3743286, "Shape__Length": 49921.5440632462}, "geometry": {"type": "Polygon", "coordinates": [[[-75.1110653346439, 40.0468227016961], [-75.1094338408832, 40.0458834049658], [-75.1092665910645, 40.0457871118939], [-75.1089076424268, 40.0461364171465], [-75.1090472451382, 40.0462188966551], [-75.1081054819242, 40.0471759196109], [-75.1082140890639, 40.0472393936405], [-75.1074295041131, 40.0480308153177], [-75.1076470759475, 40.0481486263741], [-75.1074929449898, 40.0483371581054], [-75.1068930104058, 40.0488267438584], [-75.1066723855388, 40.048702530507], [-75.1062215143984, 40.0484534566616], [-75.1056182856026, 40.0481162112071], [-75.1051137333168, 40.0478140706917], [-75.1048930526283, 40.0476490501884], [-75.1047374387325, 40.0474752837513], [-75.1046704727332, 40.047394322985], [-75.1047370863951, 40.0471839650489], [-75.10461

##### Tickets by violation

In [26]:
tickets_per_violation = session.query(Parking.violation_desc,func.count(Parking.anon_ticket_number)).\
                                    group_by(Parking.violation_desc).\
                                    order_by(func.count(Parking.anon_ticket_number).desc()).all()
tickets_per_violation

[('METER EXPIRED CC', 115063),
 ('METER EXPIRED', 55412),
 ('OVER TIME LIMIT', 52833),
 ('STOP PROHIBITED CC', 44158),
 ('EXPIRED INSPECTION', 37357),
 ('PARKING PROHBITED CC', 16571),
 ('STOPPING PROHIBITED', 14552),
 ('PARKING PROHBITED', 13035),
 ('OVER TIME LIMIT CC', 9419),
 ('PASSENGR LOADNG ZONE', 8573),
 ('FIRE HYDRANT', 6236),
 ('BUS ONLY ZONE', 6195),
 ('HP RESERVED SPACE', 5748),
 ('SIDEWALK', 4544),
 ('LOADING ZONE   CC', 4356),
 ('BUS ONLY ZONE   CC', 3856),
 ('DOUBLE PARKED  CC', 3382),
 ('CORNER CLEARANCE', 2960),
 ('SCHOOL ZONE', 2776),
 ('STREET CLEANING', 2390),
 ('CROSSWALK', 2237),
 ('HP RAMP BLOCKED', 2200),
 ('BLOCKING DRIVEWAY', 2130),
 ('SIDEWALK   CC', 1923),
 ('DOUBLE PARKED', 1839),
 ('LOADING ZONE', 1717),
 ('IMPROPER ON 2WAY HWY', 1614),
 ('PRIVATE PROPERTY', 1398),
 ('UNREG/ABANDONED VEH', 1151),
 ('EXPIRED TAG', 589),
 ('STOP MEDIAL STRIP', 564),
 ('VALET ZONE VIOLATION', 515),
 ('SNOW EMERGENCY ROUTE', 503),
 ('TAXI STAND', 450),
 ('BLOCKING DRIVEWAY CC'

In [27]:
response = {}
response['description'] = []
response['count'] = []

In [28]:
for violation in tickets_per_violation:
        response['description'].append(violation[0])
        response['count'].append(violation[1])

In [29]:
response

{'description': ['METER EXPIRED CC',
  'METER EXPIRED',
  'OVER TIME LIMIT',
  'STOP PROHIBITED CC',
  'EXPIRED INSPECTION',
  'PARKING PROHBITED CC',
  'STOPPING PROHIBITED',
  'PARKING PROHBITED',
  'OVER TIME LIMIT CC',
  'PASSENGR LOADNG ZONE',
  'FIRE HYDRANT',
  'BUS ONLY ZONE',
  'HP RESERVED SPACE',
  'SIDEWALK',
  'LOADING ZONE   CC',
  'BUS ONLY ZONE   CC',
  'DOUBLE PARKED  CC',
  'CORNER CLEARANCE',
  'SCHOOL ZONE',
  'STREET CLEANING',
  'CROSSWALK',
  'HP RAMP BLOCKED',
  'BLOCKING DRIVEWAY',
  'SIDEWALK   CC',
  'DOUBLE PARKED',
  'LOADING ZONE',
  'IMPROPER ON 2WAY HWY',
  'PRIVATE PROPERTY',
  'UNREG/ABANDONED VEH',
  'EXPIRED TAG',
  'STOP MEDIAL STRIP',
  'VALET ZONE VIOLATION',
  'SNOW EMERGENCY ROUTE',
  'TAXI STAND',
  'BLOCKING DRIVEWAY CC',
  'CORNER CLEARANCE  CC',
  'CROSSWALK    CC',
  'SNOW RELOCATION TOW',
  'IMPROPER ON 1WAY HWY',
  'CAR SHARE VEHS ONLY',
  'VALET VIOLATION',
  'COMMRCL VEH RES AREA',
  'STOP/BLOCK HIGHWAY',
  'BUS NOT IN BUS STAND',
  'DI

##### Tickets by state

In [30]:
# Get tickets by state
tickets_per_state = session.query(Parking.state, func.count(Parking.anon_ticket_number), func.avg(Parking.fine)).\
                        filter(Parking.state != 'PA').\
                        group_by(Parking.state).\
                        order_by(func.count(Parking.anon_ticket_number).desc()).\
                        order_by(func.avg(Parking.fine).desc()).all()

tickets_per_state

[('NJ', 48544, 47.722663974950564),
 ('NY', 8617, 47.67227573401416),
 ('DE', 7876, 47.858176739461655),
 ('MD', 5776, 45.54120498614959),
 ('VA', 3870, 44.54005167958656),
 ('VN', 3531, 71.92183517417162),
 ('FL', 2942, 47.709721278042146),
 ('IN', 2632, 66.5604103343465),
 ('MA', 2112, 43.687026515151516),
 ('IL', 1818, 58.93729372937294),
 ('CT', 1468, 42.822207084468666),
 ('NC', 1312, 44.24314024390244),
 ('MI', 1236, 51.82119741100323),
 ('TX', 1175, 51.42978723404255),
 ('OH', 1095, 50.12785388127854),
 ('CA', 940, 47.16489361702128),
 ('GA', 913, 46.47645125958379),
 ('AZ', 848, 48.635613207547166),
 ('DC', 644, 40.464285714285715),
 ('SC', 529, 43.986767485822305),
 ('TN', 445, 55.168539325842694),
 ('DD', 440, 83.01136363636364),
 ('MN', 368, 60.10326086956522),
 ('OK', 360, 60.458333333333336),
 ('RI', 301, 43.59136212624585),
 ('WI', 296, 52.689189189189186),
 ('ME', 292, 45.36643835616438),
 ('NH', 244, 44.91393442622951),
 ('MO', 239, 45.47698744769875),
 ('KY', 226, 45.2

In [31]:
tickets_per_state_list = {}
tickets_per_state_list['state'] = []
tickets_per_state_list['count'] = []
tickets_per_state_list['avg_fine'] = []

for i in tickets_per_state:
    tickets_per_state_list['state'].append(i[0])
    tickets_per_state_list['count'].append(i[1])
    tickets_per_state_list['avg_fine'].append(i[2])

tickets_per_state_list

{'state': ['NJ',
  'NY',
  'DE',
  'MD',
  'VA',
  'VN',
  'FL',
  'IN',
  'MA',
  'IL',
  'CT',
  'NC',
  'MI',
  'TX',
  'OH',
  'CA',
  'GA',
  'AZ',
  'DC',
  'SC',
  'TN',
  'DD',
  'MN',
  'OK',
  'RI',
  'WI',
  'ME',
  'NH',
  'MO',
  'KY',
  'CO',
  'LA',
  'AL',
  'WA',
  'VT',
  'OR',
  'WV',
  'ON',
  'TL',
  'PQ',
  'IA',
  'MS',
  'NE',
  'NM',
  'NV',
  'KS',
  'UT',
  'MT',
  'AR',
  'US',
  'ID',
  'AK',
  'ND',
  'SD',
  'WY',
  'NB',
  'HI',
  'AB',
  'NS',
  'BC',
  'MB',
  'VP',
  'PE',
  'MX',
  'SK',
  'VI',
  'JX',
  'NF',
  'YU',
  'FN',
  'PR'],
 'count': [48544,
  8617,
  7876,
  5776,
  3870,
  3531,
  2942,
  2632,
  2112,
  1818,
  1468,
  1312,
  1236,
  1175,
  1095,
  940,
  913,
  848,
  644,
  529,
  445,
  440,
  368,
  360,
  301,
  296,
  292,
  244,
  239,
  226,
  216,
  207,
  193,
  187,
  184,
  184,
  182,
  176,
  113,
  82,
  82,
  78,
  67,
  67,
  66,
  60,
  49,
  47,
  39,
  26,
  25,
  24,
  23,
  20,
  13,
  12,
  10,
  9,
  8,
  8,
 

##### Weather vs. Ticket Issuing Behavior Analysis

In [32]:
weather_type_analysis = session.query(Weather.weather_main,
                                      Weather.weather_description,
                                      func.count(Parking.anon_ticket_number),
                                      func.avg(Weather.rain_1h),
                                      func.avg(Weather.snow_3h)).\
                        filter(Parking.ymdh == Weather.ymdh).\
                        group_by(Weather.weather_main,
                                 Weather.weather_description).\
                        order_by(func.count(Parking.anon_ticket_number).desc()).all()

weather_type_analysis

[('Clouds', 'broken clouds', 102603, 0.0, 0.0),
 ('Clouds', 'overcast clouds', 90171, 0.0, 0.0),
 ('Clouds', 'scattered clouds', 90024, 0.0, 0.0),
 ('Clouds', 'few clouds', 59644, 0.0, 0.0),
 ('Clear', 'sky is clear', 28265, 0.0, 0.0),
 ('Rain', 'light rain', 26319, 0.49868688020053215, 0.02285421178616208),
 ('Mist', 'mist', 13907, 0.0, 0.0),
 ('Rain', 'moderate rain', 8955, 1.7562155220545939, 0.0),
 ('Drizzle', 'light intensity drizzle', 2585, 0.2815589941972897, 0.0),
 ('Rain', 'heavy intensity rain', 2196, 4.526780510018185, 0.0),
 ('Fog', 'fog', 1276, 0.0, 0.0),
 ('Snow', 'light snow', 1089, 0.0, 0.15670339761248833),
 ('Haze', 'haze', 815, 0.0, 0.0),
 ('Snow', 'snow', 761, 0.0, 0.36202365308804174),
 ('Thunderstorm', 'thunderstorm with rain', 390, 1.1151282051282092, 0.0),
 ('Thunderstorm', 'thunderstorm with light rain', 376, 2.949574468085108, 0.0),
 ('Thunderstorm', 'thunderstorm with heavy rain', 235, 2.7603829787234, 0.0),
 ('Thunderstorm', 'thunderstorm', 223, 0.3113901345

In [33]:
tickets_per_hour = session.query(Weather.dt,
                                 func.count(Parking.anon_ticket_number),
                                      func.avg(Weather.rain_1h),
                                      func.avg(Weather.snow_3h)).\
                        filter(Parking.ymdh == Weather.ymdh).\
                        group_by(Weather.weather_main,
                                 Weather.weather_description).\
                        order_by(func.count(Parking.anon_ticket_number).desc()).all()

In [34]:
test = engine.execute('SELECT * FROM weather JOIN ppa ON weather.ymdh=ppa.ymdh').fetchall()

In [35]:
type(test)

list

In [95]:
tickets_per_hour = session.query(Weather.dt,
                     Weather.month,
                     Weather.day,
                     Weather.hour,
                     Weather.weather_id,
                     Weather.weather_main,
                     Weather.weather_description,
                     func.count(Parking.anon_ticket_number)).\
                    filter(Parking.ymdh == Weather.ymdh).\
                    group_by(Weather.dt).\
                    order_by(Weather.weather_id.asc()).all()
#                     order_by(Weather.dt.asc()).all()

In [96]:
tickets_per_hour_list = {}
tickets_per_hour_list['datetime'] = []
tickets_per_hour_list['month'] = []
tickets_per_hour_list['day'] = []
tickets_per_hour_list['hour'] = []
tickets_per_hour_list['weather_id'] = []
tickets_per_hour_list['weather_main'] = []
tickets_per_hour_list['weather_description'] = []
tickets_per_hour_list['anon_ticket_number'] = []

for i in tickets_per_hour:
    tickets_per_hour_list['datetime'].append(i[0])
    tickets_per_hour_list['month'].append(i[1])
    tickets_per_hour_list['day'].append(i[2])
    tickets_per_hour_list['hour'].append(i[3])
    tickets_per_hour_list['weather_id'].append(i[4])
    tickets_per_hour_list['weather_main'].append(i[5])
    tickets_per_hour_list['weather_description'].append(i[6])
    tickets_per_hour_list['anon_ticket_number'].append(i[7])                  

tickets_per_hour_list

{'datetime': [datetime.datetime(2017, 2, 25, 22, 0),
  datetime.datetime(2017, 3, 28, 13, 0),
  datetime.datetime(2017, 3, 31, 3, 0),
  datetime.datetime(2017, 4, 6, 16, 0),
  datetime.datetime(2017, 6, 19, 22, 0),
  datetime.datetime(2017, 6, 27, 7, 0),
  datetime.datetime(2017, 8, 5, 8, 0),
  datetime.datetime(2017, 5, 25, 22, 0),
  datetime.datetime(2017, 7, 24, 7, 0),
  datetime.datetime(2017, 8, 18, 12, 0),
  datetime.datetime(2017, 9, 6, 10, 0),
  datetime.datetime(2017, 4, 6, 19, 0),
  datetime.datetime(2017, 7, 14, 20, 0),
  datetime.datetime(2017, 7, 25, 1, 0),
  datetime.datetime(2017, 8, 2, 21, 0),
  datetime.datetime(2017, 8, 23, 4, 0),
  datetime.datetime(2017, 4, 22, 1, 0),
  datetime.datetime(2017, 4, 29, 9, 0),
  datetime.datetime(2017, 6, 14, 13, 0),
  datetime.datetime(2017, 8, 2, 17, 0),
  datetime.datetime(2017, 8, 5, 7, 0),
  datetime.datetime(2017, 1, 2, 22, 0),
  datetime.datetime(2017, 1, 3, 4, 0),
  datetime.datetime(2017, 1, 3, 21, 0),
  datetime.datetime(2017

In [97]:
tickets_per_hour

[(datetime.datetime(2017, 2, 25, 22, 0),
  2,
  25,
  22,
  '200',
  'Thunderstorm',
  'thunderstorm with light rain',
  31),
 (datetime.datetime(2017, 3, 28, 13, 0),
  3,
  28,
  13,
  '200',
  'Thunderstorm',
  'thunderstorm with light rain',
  145),
 (datetime.datetime(2017, 3, 31, 3, 0),
  3,
  31,
  3,
  '200',
  'Thunderstorm',
  'thunderstorm with light rain',
  1),
 (datetime.datetime(2017, 4, 6, 16, 0),
  4,
  6,
  16,
  '200',
  'Thunderstorm',
  'thunderstorm with light rain',
  110),
 (datetime.datetime(2017, 6, 19, 22, 0),
  6,
  19,
  22,
  '200',
  'Thunderstorm',
  'thunderstorm with light rain',
  1),
 (datetime.datetime(2017, 6, 27, 7, 0),
  6,
  27,
  7,
  '200',
  'Thunderstorm',
  'thunderstorm with light rain',
  39),
 (datetime.datetime(2017, 8, 5, 8, 0),
  8,
  5,
  8,
  '200',
  'Thunderstorm',
  'thunderstorm with light rain',
  49),
 (datetime.datetime(2017, 5, 25, 22, 0),
  5,
  25,
  22,
  '201',
  'Thunderstorm',
  'thunderstorm with rain',
  27),
 (dateti

In [40]:
len(tickets_per_hour)

8397

In [90]:
ticket_count = session.query(Weather.weather_id,
                             Weather.weather_main,
                             Weather.weather_description,
                       func.count(Parking.anon_ticket_number)).\
                       filter(Parking.ymdh == Weather.ymdh).\
                       group_by(Weather.weather_id).\
                       order_by(Weather.weather_id.asc()).all()
ticket_count

[('200', 'Thunderstorm', 'thunderstorm with light rain', 376),
 ('201', 'Thunderstorm', 'thunderstorm with rain', 390),
 ('202', 'Thunderstorm', 'thunderstorm with heavy rain', 235),
 ('211', 'Thunderstorm', 'proximity thunderstorm', 333),
 ('300', 'Drizzle', 'light intensity drizzle', 2585),
 ('500', 'Rain', 'light rain', 26319),
 ('501', 'Rain', 'moderate rain', 8955),
 ('502', 'Rain', 'heavy intensity rain', 2196),
 ('503', 'Rain', 'very heavy rain', 111),
 ('511', 'Rain', 'freezing rain', 28),
 ('600', 'Snow', 'light snow', 1089),
 ('601', 'Snow', 'snow', 761),
 ('602', 'Snow', 'heavy snow', 48),
 ('701', 'Mist', 'mist', 13907),
 ('721', 'Haze', 'haze', 815),
 ('741', 'Fog', 'fog', 1276),
 ('800', 'Clear', 'sky is clear', 28265),
 ('801', 'Clouds', 'few clouds', 59644),
 ('802', 'Clouds', 'scattered clouds', 90024),
 ('803', 'Clouds', 'broken clouds', 102603),
 ('804', 'Clouds', 'overcast clouds', 90171)]

In [91]:
hour_count = session.query(Weather.weather_id,
                           Weather.weather_main,
                           Weather.weather_description,
                     func.count(Weather.weather_description)).\
                     group_by(Weather.weather_id).\
                     order_by(Weather.weather_id.asc()).all()
hour_count

[('200', 'Thunderstorm', 'thunderstorm with light rain', 7),
 ('201', 'Thunderstorm', 'thunderstorm with rain', 4),
 ('202', 'Thunderstorm', 'thunderstorm with heavy rain', 5),
 ('211', 'Thunderstorm', 'proximity thunderstorm', 5),
 ('300', 'Drizzle', 'light intensity drizzle', 48),
 ('500', 'Rain', 'light rain', 525),
 ('501', 'Rain', 'moderate rain', 178),
 ('502', 'Rain', 'heavy intensity rain', 48),
 ('503', 'Rain', 'very heavy rain', 1),
 ('511', 'Rain', 'freezing rain', 2),
 ('600', 'Snow', 'light snow', 49),
 ('601', 'Snow', 'snow', 33),
 ('602', 'Snow', 'heavy snow', 1),
 ('701', 'Mist', 'mist', 227),
 ('721', 'Haze', 'haze', 11),
 ('741', 'Fog', 'fog', 19),
 ('800', 'Clear', 'sky is clear', 805),
 ('801', 'Clouds', 'few clouds', 1184),
 ('802', 'Clouds', 'scattered clouds', 1741),
 ('803', 'Clouds', 'broken clouds', 1967),
 ('804', 'Clouds', 'overcast clouds', 1900)]

In [64]:
# ticket_count[0][2]

376

In [92]:
# Calculate the average tickets per hour based on each weather type
ticket_count_list = []
for i in ticket_count:
    ticket_count_list.append(i[3])

hour_count_list = []
for i in hour_count:
    hour_count_list.append(i[3])
    
avg_count_list = []
for i in range(len(ticket_count_list)):
    avg_count_list.append(ticket_count_list[i]/hour_count_list[i])
    
avg_count_list

[53.714285714285715,
 97.5,
 47.0,
 66.6,
 53.854166666666664,
 50.13142857142857,
 50.30898876404494,
 45.75,
 111.0,
 14.0,
 22.224489795918366,
 23.060606060606062,
 48.0,
 61.26431718061674,
 74.0909090909091,
 67.15789473684211,
 35.11180124223603,
 50.375,
 51.708213670304424,
 52.16217590238943,
 47.45842105263158]

In [94]:
avg_tickets_per_weather_type = {}

avg_tickets_per_weather_type['weather_id'] = []
avg_tickets_per_weather_type['weather_main'] = []
avg_tickets_per_weather_type['weather_description'] = []
avg_tickets_per_weather_type['ticket_count'] = []

for index, data in enumerate(ticket_count):
    avg_tickets_per_weather_type['weather_id'].append(data[0])
    avg_tickets_per_weather_type['weather_main'].append(data[1])
    avg_tickets_per_weather_type['weather_description'].append(data[2])
    avg_tickets_per_weather_type['ticket_count'].append(round(avg_count_list[index], 1))

avg_tickets_per_weather_type

{'weather_id': ['200',
  '201',
  '202',
  '211',
  '300',
  '500',
  '501',
  '502',
  '503',
  '511',
  '600',
  '601',
  '602',
  '701',
  '721',
  '741',
  '800',
  '801',
  '802',
  '803',
  '804'],
 'weather_main': ['Thunderstorm',
  'Thunderstorm',
  'Thunderstorm',
  'Thunderstorm',
  'Drizzle',
  'Rain',
  'Rain',
  'Rain',
  'Rain',
  'Rain',
  'Snow',
  'Snow',
  'Snow',
  'Mist',
  'Haze',
  'Fog',
  'Clear',
  'Clouds',
  'Clouds',
  'Clouds',
  'Clouds'],
 'weather_description': ['thunderstorm with light rain',
  'thunderstorm with rain',
  'thunderstorm with heavy rain',
  'proximity thunderstorm',
  'light intensity drizzle',
  'light rain',
  'moderate rain',
  'heavy intensity rain',
  'very heavy rain',
  'freezing rain',
  'light snow',
  'snow',
  'heavy snow',
  'mist',
  'haze',
  'fog',
  'sky is clear',
  'few clouds',
  'scattered clouds',
  'broken clouds',
  'overcast clouds'],
 'ticket_count': [53.7,
  97.5,
  47.0,
  66.6,
  53.9,
  50.1,
  50.3,
  45.8,
 

In [98]:
session.query(Parking.violation_desc, func.max(Parking.fine)).\
            group_by(Parking.violation_desc).\
            order_by(func.max(Parking.fine).desc()).all()

[('FRAUD PARK HP SPACE', 1001),
 ('COUNTERFEIT HP PERM', 1001),
 ('WASH/REPAIR VEH', 301),
 ('UNREG/ABANDONED VEH', 301),
 ('OWNER ID ON COM VEH', 301),
 ('INVALID RPP PERMIT', 301),
 ('INVALID KIOSK RECPT', 301),
 ('INVALID CONT PERMIT', 301),
 ('HP RESERVED SPACE', 301),
 ('DISPLAY VEH FOR SALE', 301),
 ('BUS-IMP PSNG DISCHRG', 251),
 ('BUS NOT IN BUS STAND', 251),
 ('VALET VIOLATION', 101),
 ('UNLAWFUL ALARM', 101),
 ('SUSPENDED LICENSE', 101),
 ('PKG W/LIC SUSPD 75', 101),
 ('PARKED ON GRASS', 101),
 ('PARENTAL LIABILITY', 101),
 ('EXCESSIVE NOISE', 101),
 ('EXCESSIVE IDLING', 101),
 ('DAMAGE TO METER', 101),
 ('COMMRCL VEH RES AREA', 101),
 ('BLOCKNG MASS TRANSIT', 101),
 ('STOP/BLOCK HIWY  CC', 76),
 ('STOP PROHIBITED CC', 76),
 ('SNOW RELOCATION TOW', 76),
 ('SIDEWALK   CC', 76),
 ('PARK PROHIB PLACE', 76),
 ('HP RAMP BLOCKED', 76),
 ('FIRE HYDRANT', 76),
 ('DOUBLE PARKED  CC', 76),
 ('CROSSWALK    CC', 76),
 ('CORNER CLEARANCE  CC', 76),
 ('BUS ONLY ZONE   CC', 76),
 ('BLOCKING